In [ ]:
# app/main.py
from fastapi import FastAPI, Depends, HTTPException
from sqlalchemy.orm import Session
from starlette.graphql import GraphQLApp
import graphene

from . import models, schemas, crud
from .database import SessionLocal, engine, Base

Base.metadata.create_all(bind=engine)

app = FastAPI(title="Bank Branch API")

def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# REST endpoints

@app.get("/banks", response_model=list[schemas.Bank])
def read_banks(skip: int = 0, limit: int = 100, db: Session = Depends(get_db)):
    return crud.get_banks(db, skip=skip, limit=limit)

@app.get("/banks/{bank_id}/branches", response_model=list[schemas.Branch])
def read_branches_for_bank(bank_id: int, skip: int = 0, limit: int = 100, db: Session = Depends(get_db)):
    bank = crud.get_bank(db, bank_id)
    if not bank:
        raise HTTPException(status_code=404, detail="Bank not found")
    return crud.get_branches_for_bank(db, bank_id, skip, limit)

@app.get("/branches/{ifsc}", response_model=schemas.Branch)
def read_branch_by_ifsc(ifsc: str, db: Session = Depends(get_db)):
    branch = crud.get_branch_by_ifsc(db, ifsc)
    if not branch:
        raise HTTPException(status_code=404, detail="Branch not found")
    return branch

# GraphQL schema using graphene

class BankType(graphene.ObjectType):
    id = graphene.Int()
    bank_id = graphene.Int()
    name = graphene.String()

class BranchType(graphene.ObjectType):
    id = graphene.Int()
    ifsc = graphene.String()
    branch = graphene.String()
    address = graphene.String()
    city = graphene.String()
    district = graphene.String()
    state = graphene.String()
    bank = graphene.Field(BankType)

class Query(graphene.ObjectType):
    branches = graphene.List(BranchType)
    branch = graphene.Field(BranchType, ifsc=graphene.String(required=True))

    def resolve_branches(self, info):
        db = SessionLocal()
        try:
            items = db.query(models.Branch).limit(1000).all()
            return items
        finally:
            db.close()

    def resolve_branch(self, info, ifsc):
        db = SessionLocal()
        try:
            return db.query(models.Branch).filter(models.Branch.ifsc == ifsc).first()
        finally:
            db.close()

schema = graphene.Schema(query=Query)

app.add_route("/gql", GraphQLApp(schema=schema))   # GraphQL at /gql